In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.enable_eager_execution()

In [3]:
data = pd.read_csv("wine.csv")
feature_columns = ['Alcohol', 'Malic.acid', 'Ash', 'Acl', 'Mg', 'Phenols',
       'Flavanoids', 'Nonflavanoid.phenols', 'Proanth', 'Color.int', 'Hue',
       'OD', 'Proline']
data["Wine"] = data["Wine"] - 1  # original unique value [1, 2, 3]
data.head()

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,0,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,0,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
Wine                    178 non-null int64
Alcohol                 178 non-null float64
Malic.acid              178 non-null float64
Ash                     178 non-null float64
Acl                     178 non-null float64
Mg                      178 non-null int64
Phenols                 178 non-null float64
Flavanoids              178 non-null float64
Nonflavanoid.phenols    178 non-null float64
Proanth                 178 non-null float64
Color.int               178 non-null float64
Hue                     178 non-null float64
OD                      178 non-null float64
Proline                 178 non-null int64
dtypes: float64(11), int64(3)
memory usage: 19.5 KB


In [13]:
X = data[feature_columns].values
y = data["Wine"].values

onc = OneHotEncoder(sparse=False)
Y = onc.fit_transform(y.reshape(-1, 1))

X, Y = shuffle(X, Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)


In [14]:
epochs = 100
display_step = 10

In [27]:
W = tf.get_variable(name="W", shape=[13, 3], dtype=tf.float32, initializer=tf.zeros_initializer)
b = tf.get_variable(name="b", shape=[1, 3], dtype=tf.float32, initializer=tf.zeros_initializer)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-1)
variables = [W, b]

def model(X, W, b):
    logits = tf.matmul(X, W) + b
    return logits
    

for epoch_id in range(epochs):
    with tf.GradientTape() as tape:
        logits = model(tf.convert_to_tensor(X_train, dtype=tf.float32), W, b)
        loss = tf.losses.softmax_cross_entropy(onehot_labels=Y_train, logits=logits)
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(grads_and_vars=zip(gradients, variables))
    if epoch_id % display_step == 0:
        pred_logits = model(tf.convert_to_tensor(X_test, dtype=tf.float32), W, b)
        y_pred = tf.argmax(pred_logits, axis=1).numpy()
        accuracy = sum(y_pred == Y_test.argmax(axis=1)) / len(Y_test)
        print("epochID : %d testAcc: %.4f trainLoss: %.4f" % (epoch_id, accuracy, loss))
        

epochID : 0 testAcc: 0.9333 trainLoss: 1.0986
epochID : 10 testAcc: 0.9556 trainLoss: 0.3937
epochID : 20 testAcc: 0.9556 trainLoss: 0.2582
epochID : 30 testAcc: 0.9556 trainLoss: 0.2002
epochID : 40 testAcc: 0.9556 trainLoss: 0.1670
epochID : 50 testAcc: 0.9556 trainLoss: 0.1452
epochID : 60 testAcc: 0.9778 trainLoss: 0.1296
epochID : 70 testAcc: 1.0000 trainLoss: 0.1178
epochID : 80 testAcc: 1.0000 trainLoss: 0.1084
epochID : 90 testAcc: 1.0000 trainLoss: 0.1008
